# Introduction to the Citrination Python API

*Authors: Eric Muckley*

This notebook enables upload and storage of data to Citrine's Citrination web application using the Citrination Python API. Workflow of the code:
* A connection to the Citrination client web API is established
* A new dataset is created on Citrination
* Data is converted into PIF format and exported as a *json* file
* The *json* file containing the data is uploaded to the new dataset on Citrination
* A data view is created to enable exploration of the uploaded data

## Install the Citrination client for Python

Before using the Citrination Python API, the `citrination-client` library must be installed in Python. Documentation for the library can be found [here](https://citrineinformatics.github.io/python-citrination-client/index.html). To install the library using Anaconda, use `conda install -c conda-forge citrination-client` as described [here](https://anaconda.org/conda-forge/citrination-client). Detailed information about the `citrination-client` library, tutorials, and additional help can be found in the [learn-citrination Github repository](https://github.com/CitrineInformatics/learn-citrination).

### Getting the Citrination API key

To access Citrination using the API, an API key is required. The instructions are described in more detail in the [learn-citrination Github repository](https://github.com/CitrineInformatics/learn-citrination). To get an API key:
1. Log into https://citrination.com
2. Navigate to your `Username` in the upper right-hand corner of the window
3. Navigate to `Account settings` → `Show API key`
4. Click `Show API key`, and copy the key

### Setting the API key as an environment variable

Once the API key is copied, it can be used in the Python code to connect to the Citrination API. This can be done by setting the API key as an environmental variable by adding something like `export CITRINATION_API_KEY="YOUR_API_KEY_HERE"` inside your .bash_profile (recommended), or simply pasting the API key directly into the Python code below. To set the API key as an environmental variable on MacOS, edit the `.bash_profile` file by opening a terminal and typing `touch ~/.bash_profile; open ~/.bash_profile`. At the end of the file, add this line to set the API key: ```export CITRINATION_API_KEY="YOUR_API_KEY_HERE"```.  

These instructions are described in more detail in the [learn-citrination Github repository](https://github.com/CitrineInformatics/learn-citrination).

In [ ]:
! pip install citrination
! pip install citrination_client

In [ ]:
import os
import csv
import time
import numpy as np
import pandas as pd

import citrination_client
from citrination_client.views.client import DataViewsClient
from citrination_client.views.data_view_builder import DataViewBuilder
from citrination_client import CategoricalDescriptor, InorganicDescriptor, RealDescriptor

### Connect to the Citrination client

Create the citrination `client` object for communication with the Citrination API. To specify the API key, use the environment variable which was set in `~/.bash_profile` (recommended), or directly paste the API key as a string here.

In [ ]:
api_key = os.environ['CITRINATION_API_KEY']
# OR: api_key = "YOUR_API_KEY_HERE"

client = citrination_client.CitrinationClient(
        api_key=api_key, site='https://citrination.com')

### Create new dataset on Citrination

All data uploaded to Citrination must be uploaded to a specfic dataset. Once a dataset is created in Citrination, a user can upload as many files to the dataset as desired. Attempting to create a dataset which already exists will result in an error. A new dataset can be created on Citrination using the [web app](https://citrination.com) or the Python API. Details about creating a new dataset can be found [here](http://citrineinformatics.github.io/python-citrination-client/tutorial/data_examples.html#checking-the-ingest-status-of-a-dataset).

Each dataset can be set as public (publicly searchable and viewable) or private (viewable only to the user who uploaded it). This permission is controlled by setting `public=True` or `public=False` in the `create_dataset` method. More details related to dataset manipulation can be found [here](http://citrineinformatics.github.io/python-citrination-client/tutorial/data_examples.html#dataset-manipulation).

The ID number of each dataset is used to identify the dataset for the purposes of uploading, downloading, searching, or manipulating the dataset. To access the ID of a new dataset, use `dataset.id` in the API. To access the `dataset.id` of an existing dataset using the Citrination web application:
1. Log into https://citrination.com/
2. Navigate to `Datasets`
3. Navigate to `My Datasets`
4. Copy the ID number of the dataset you want to upload to

In [ ]:
# specify name of new dataset to be created on Citrination
dataset_name = 'Our new dataset'
# add description of the dataset to display on Citrination
dataset_description = 'Contains data uploaded during TAMU school 2020'

# create the new dataset and allow public access
try:
    dataset = client.data.create_dataset(dataset_name, dataset_description, public=True)
    dataset_id = dataset.id
    print('Dataset "{}" created with dataset ID: {}'.format(new_dataset_name, dataset_id))
except:
    print('This dataset cannot be created.')
    print(r'Please check https://citrination.com to see if it already exists, or change its name.')

## Get some data to upload

Now we need some data to upload to Citrination. We take an example dataset of materials proeperties which was downloaded from Wikipedia.

For convienence, all the dta is embedded inside this notebook. Typically, this data would be imported or copied from an external file or database.

In [ ]:

data = {
    'name': ['Zinc bromide', 'Benzo(a)fluorene', 'Lead(II) iodide', 'Lead(II) oxide', 'Strontium fluoride', 'Copper(I) chloride', 'Vanadium(V) oxide', 'Trichloroethylene', 'Copper(II) perchlorate', 'Cobalt(II) fluoride', 'Sodium iodide', 'Methyl isopropyl ketone', 'Chlorotrifluoromethane', 'Zinc sulfate', 'Radium bromide', 'Sodium hypochlorite', 'Cyclopentane', 'Mercury(II) iodide', 'Lithium hydroxide', 'Lead(II) fluoride'],
    'formula': ['ZnBr$_{2}$', 'C$_{17}$H$_{12}$', 'PbI$_{2}$', 'PbO', 'SrF$_{2}$', 'CuCl', 'V$_{2}$O$_{5}$', 'C$_{2}$HCl$_{3}$', 'Cu(ClO$_{4}$)$_{2}$', 'CoF$_{2}$', 'NaI', 'C$_{5}$H$_{10}$O', 'CClF$_{3}$', 'ZnSO$_{4}$', 'RaBr$_{2}$', 'NaClO', 'C$_{5}$H$_{10}$', 'HgI$_{2}$', 'LiOH', 'PbF$_{2}$'],
    'molarmass': ['225.198', '216.277', '461.01', '223.20', '125.62', '98.999', '181.8800', '141.4', '262.447', '96.93', '149.89', '86.13', '104.46', '161.47', '385.782', '74.442', '70.1', '454.40', '23.95', '245.20'],
    'meltingpoint': ['394', '189.5', '402', '888', '1477', '426', '690', '-73', '82', '1217', '661', '-92', '-181', '680', '728', '18', '-93.9', '259', '462', '824'],
    'density': ['4.20', '0.819', '6.16', '9.53', '4.24', '4.145', '3.357', '1.46', '2.225', '4.46', '3.67', '0.803', '1.526', '3.54', '5.79', '1.11', '0.751', '6.36', '1.46', '8.445'],
    'boilingpoint': ['697', '405', '872', '1477', '2460', '1490', '1750', '87.2', '120', '1400', '1304', '92', '-81.5', '740', '900', '101', '49.2', '350', '924', '1293'],
    'solubility': ['311', '0.000045', '0.044', '0.017', '0.039', '0.0062', '0.8', '1.280', '146', '1.4', '158.7', '6', '0.009', '57.7', '70.6', '29.3', '156', '0.006', '12.7', '0.057']}



df = pd.DataFrame(data)

print('Data shape: {} rows, {} columns'.format(df.shape[0], df.shape[1]))
df.head(20)

Data shape: 20 rows, 7 columns


name              formula  ... boilingpoint solubility
0              Zinc bromide           ZnBr$_{2}$  ...          697        311
1          Benzo(a)fluorene     C$_{17}$H$_{12}$  ...          405   0.000045
2           Lead(II) iodide            PbI$_{2}$  ...          872      0.044
3            Lead(II) oxide                  PbO  ...         1477      0.017
4        Strontium fluoride            SrF$_{2}$  ...         2460      0.039
5        Copper(I) chloride                 CuCl  ...         1490     0.0062
6         Vanadium(V) oxide       V$_{2}$O$_{5}$  ...         1750        0.8
7         Trichloroethylene     C$_{2}$HCl$_{3}$  ...         87.2      1.280
8    Copper(II) perchlorate  Cu(ClO$_{4}$)$_{2}$  ...          120        146
9       Cobalt(II) fluoride            CoF$_{2}$  ...         1400        1.4
10            Sodium iodide                  NaI  ...         1304      158.7
11  Methyl isopropyl ketone     C$_{5}$H$_{10}$O  ...           92          6
12   Chlorotrifluoromethane           CClF$_{3}$  ...        -81.5      0.009
13             Zinc sulfate           ZnSO$_{4}$  ...          740       57.7
14           Radium bromide           RaBr$_{2}$  ...          900       70.6
15      Sodium hypochlorite                NaClO  ...          101       29.3
16             Cyclopentane      C$_{5}$H$_{10}$  ...         49.2        156
17       Mercury(II) iodide            HgI$_{2}$  ...          350      0.006
18        Lithium hydroxide                 LiOH  ...          924       12.7
19        Lead(II) fluoride            PbF$_{2}$  ...         1293      0.057

[20 rows x 7 columns]

## Prepare data for conversion to PIF format

Data is stored on Citrination using the [property information file (PIF) format](https://citrineinformatics.github.io/pif-documentation/). To get data into PIF-compatible format, we use the [pypif](https://github.com/CitrineInformatics/pypif) library, which can be installed using `pip install pypif`. Once data is in PIF format, we export it as a *json* file so it can be uploaded by Citrination. 

Uploading data that is in PIF-compatile format makes it easy for CItrination to ingest the data as it isuploaded. This makes it searchable, organized, nd structured on the Citrination web app.

In [ ]:
from pypif import pif
from pypif.obj import ChemicalSystem, Property

def to_pif_file(data, filepath, print_pif=False):
    """Export data in PIF format to a json file for uploading to Citrination"""
    pif_data = pif.dumps(data, indent=2)
    with open(filepath, "w") as f:
        f.write(pif_data)
    if print_pif:
        print(pif_data)

## Add user-defined details for data upload

While structuring the data in PIF format, we can add custom user-defined parameters. These will also be searchable and viewable on Citrination.

In [ ]:
# specify experiment details
experiment_id = 'Experiment_2020-06-29'        # unique experiment name or ID
instrument_name = 'Instrument XYZ'             # instrument name
contact_name = 'Eric M'                        # name of person to contact
contact_email = 'em@email.com'                 # email of person to contact
notes = [                                      # user-defined notes in list format
    'Custom note #1',
    'Custom note #2',
]

## Convert the data to PIF format

In [ ]:
# get all properties to add to sample
property_names = [
    'molarmass',
    'meltingpoint',
    'density',
    'boilingpoint',
    'solubility']

sample_list = [] 

# loop over each sample and add to PIF json
for i in range(len(df)):

    # create new sample and populate experiment details
    cs = ChemicalSystem()
    cs.uid = experiment_id+'_record_'+str(i).zfill(5)
    cs.names = df['name'].iloc[i]
    cs.instrument = instrument_name
    cs.chemical_formula = df['formula'].iloc[i]
    cs.tags = notes
    cs.contact = {
        'name': contact_name,
        'email': contact_email}

    property_list = []
    # loop over each property
    for prop_name in property_names:
        p = Property()
        p.name = prop_name
        p.scalars = df[prop_name].iloc[i]
        property_list.append(p)


    cs.properties = property_list

    # append sample to list of samples
    sample_list.append(cs)

# save data to PIF-formatted json file for uploading to Citrination
pif_data_filename = 'pif_data.json'
to_pif_file(sample_list, pif_data_filename, print_pif=True)

### Upload data to Citrination

To upload files to a dataset on Citrination, we use the `dataset_id` which was copied from the [Citrination web app](https://citrination.com/datasets), or from the `dataset.id` of a newly created dataset using the API. 

Once the uploaded data is processed by Citrination, you may navigate to [Citrination](https://citrination.com) and click `Datasets` and `My datasets` to view your dataset. After selecting your dataset scroll to the bottom of the dataset page and click on one of the data cards to view the ingested data which contains custom user input information, the linked experimental data and formatted experimental details.

In [ ]:
# upload json file of experimental data in PIF format
print(client.data.upload(dataset_id, pif_data_filename))
print('Uploaded {} properties of {} samples to dataset ID#{}'.format(
        df.shape[1], df.shape[0], dataset_id))

# wait until upload is finished processing
status = 'Processing'
while status == 'Processing':
    current_time = datetime.now().strftime('%m-%d-%Y_%H-%M-%S')
    status = client.data.get_ingest_status(dataset_id)
    print('Time: {}. Upload status: {}'.format(current_time, status))
    time.sleep(5)

## Create a dataview

To view all the data in a dataset on Citrination, and to enable machine learning and pltoting of the data on Citrination, we add the data to a *Data View*. First, initialize the data view client, and create a new data view using our dataset ID.

### Connect to data view client

In [ ]:
# create data view client
dv_client = DataViewsClient(os.environ["CITRINATION_API_KEY"], 'https://citrination.com')

# get labels for identifying the data view
dv_name = 'View for {}'.format(dataset_name)
dv_description = 'View for {} dataset'.format(dataset_name)

### Build a default data view

This is a quick way to create a dataview which uses default parameters.

In [ ]:
# get data view configuration object
dv_config = dv_client.create_ml_configuration_from_datasets([str(dataset_id)])
print(dv_config)
dv_id = dv_client.create(dv_config, dv_name, dv_description)

## Viewing the data on Citrination

To view the uploaded data,
1. Log in to https://citrination.com
2. On the top menubar, click **_Datasets_**, then **_My datasets_**
3. Click on the dataset which was just uploaded
4. Scroll to the bottom of the page and click on a data card to see how the data was uploaded
5. In the data set page, click **_Files_** to view or download all the original files which were oploaded with the data

To plot data and create machine learning models,
1. Navigate to **_Data Views_** on the top menubar
2. Scroll to the bottom of the page and click on the data view
3. Inside the data view, navigate to **_Matrix_**
4. Navigate to **_General_** and **_Plot_** to view and plot data